In [ ]:
from web3 import Web3
import time
import subprocess
import sys
from google.cloud import storage
import os
import pandas as pd

In [ ]:
with open("rpc.txt", "r") as file:
    rpc = file.read().strip()
w3 = Web3(Web3.HTTPProvider(rpc))

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './ethereum-data-nero.json'

In [ ]:
abi = """[
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": true,
				"internalType": "uint256",
				"name": "schemeId",
				"type": "uint256"
			},
			{
				"indexed": true,
				"internalType": "address",
				"name": "stealthAddress",
				"type": "address"
			},
			{
				"indexed": false,
				"internalType": "bytes",
				"name": "ephemeralPubKey",
				"type": "bytes"
			},
			{
				"indexed": false,
				"internalType": "bytes",
				"name": "metadata",
				"type": "bytes"
			}
		],
		"name": "Announcement",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "schemeId",
				"type": "uint256"
			},
			{
				"internalType": "address",
				"name": "stealthAddress",
				"type": "address"
			},
			{
				"internalType": "bytes",
				"name": "ephemeralPubKey",
				"type": "bytes"
			},
			{
				"internalType": "bytes",
				"name": "metadata",
				"type": "bytes"
			}
		],
		"name": "announce",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	}
]"""

abi = abi.replace("\n", "").replace("\t", "")

In [ ]:
bytecode = "608060405234801561001057600080fd5b5061041c806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c80634d1f958314610030575b600080fd5b61004a60048036038101906100459190610291565b61004c565b005b8273ffffffffffffffffffffffffffffffffffffffff16847fc161b5d8d13c524d769545162d90b3e4341d89659fa54b3126825f4cabc2658284846040516100959291906103af565b60405180910390a350505050565b6000604051905090565b600080fd5b600080fd5b6000819050919050565b6100ca816100b7565b81146100d557600080fd5b50565b6000813590506100e7816100c1565b92915050565b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b6000610118826100ed565b9050919050565b6101288161010d565b811461013357600080fd5b50565b6000813590506101458161011f565b92915050565b600080fd5b600080fd5b6000601f19601f8301169050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b61019e82610155565b810181811067ffffffffffffffff821117156101bd576101bc610166565b5b80604052505050565b60006101d06100a3565b90506101dc8282610195565b919050565b600067ffffffffffffffff8211156101fc576101fb610166565b5b61020582610155565b9050602081019050919050565b82818337600083830152505050565b600061023461022f846101e1565b6101c6565b9050828152602081018484840111156102505761024f610150565b5b61025b848285610212565b509392505050565b600082601f8301126102785761027761014b565b5b8135610288848260208601610221565b91505092915050565b600080600080608085870312156102ab576102aa6100ad565b5b60006102b9878288016100d8565b94505060206102ca87828801610136565b935050604085013567ffffffffffffffff8111156102eb576102ea6100b2565b5b6102f787828801610263565b925050606085013567ffffffffffffffff811115610318576103176100b2565b5b61032487828801610263565b91505092959194509250565b600081519050919050565b600082825260208201905092915050565b60005b8381101561036a57808201518184015260208101905061034f565b60008484015250505050565b600061038182610330565b61038b818561033b565b935061039b81856020860161034c565b6103a481610155565b840191505092915050565b600060408201905081810360008301526103c98185610376565b905081810360208301526103dd8184610376565b9050939250505056fea26469706673582212200deb5709652449ae0ca8362005471cfd975c503ff16802cae6b745de564b045e64736f6c63430008120033"

In [ ]:
announcementContractAddress = "0x2683f8174496420346B699Bae3AF637db8E1E46c"

In [ ]:
messenger = w3.eth.contract(abi=abi, bytecode=bytecode)

In [ ]:
announcement_filter = messenger.events.Announcement.createFilter(
    fromBlock="latest",  
    address= announcementContractAddress)

In [ ]:
client = storage.Client()

In [ ]:
def optimize_announcements():
    df = pd.read_csv("announcements.csv")
    df_c = df.groupby("sender")["metadata"].count().reset_index().rename(columns={"metadata": "senderCount"})
    df = pd.merge(df, df_c, on="sender", how="left")
    df = df.sort_values(["senderCount", "blockNumber"], ascending=[True, False])
    df.drop("senderCount", axis=1).to_csv("announcements.csv", index=None)

In [ ]:
while True:
    _a = announcement_filter.get_new_entries()
    if len(_a) > 0:
        announcements = []
        for announcement in _a:
            a = announcement["args"]
            txhash = announcement["transactionHash"].hex()
            blocknr = int(announcement["blockNumber"])
            tx = w3.eth.get_transaction(txhash)
            sender = tx["from"]
            announcements.append(
                (a["schemeId"], 
                 a["stealthAddress"].lower(), 
                 a["ephemeralPubKey"].hex(), 
                 a["metadata"].hex(),
                 blocknr,
                 sender.lower()
                )
            )
            
            
        with open("announcements.csv", "a") as file:
            for a in announcements:
                content = '{},{},0x{},0x{},{},{}'.format(
                    a[0],
                    a[1],
                    a[2],
                    a[3],
                    a[4],
                    a[5]
                )
                file.write(content + "\n")
                print(f'new entry: {a[1]}')
                
            
        optimize_announcements()
        
        #subprocess.call(["./update_announcements.sh"])
        with open("announcements.csv", "r") as file:
            f = file.read()
        bucket = client.get_bucket("eip5564_data")
        blob = bucket.get_blob("announcements.csv")
        blob.upload_from_string(f)
        
    time.sleep(8)